This notebook is to evaluate the designated Softmax Model trained on the 30 Species

In [ ]:
from src import ds_generator,visualizer
import tensorflow as tf
import util

In [ ]:
train_ds, val_ds, test_ds, df = ds_generator.DS_Generator().generate_species_data(util.TRAIN_SPECIES_DF,
                                                                                  augment=1,
                                                                                  batch_size=64,
                                                                                  seed=util.SPECIES_SEED,
                                                                                  return_eval_data=True)
test_df = df[df["which_set"] == "test_ds"]
## Load Best Performing model
model = tf.keras.models.load_model("Saved Models/inception_v3_max_pooling_imagenet_weights")
# compile it with the metrics you want
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss="categorical_crossentropy",
              metrics=["acc", tf.keras.metrics.TopKCategoricalAccuracy(
                  k=3, name='top_3_categorical_accuracy'
              )])
# evaluate it
model.evaluate(test_ds, batch_size=64)

In [ ]:
# calculate test embeddings and get top 3 highest predictions
predicted_test_data = model.predict(test_ds.batch(64))
best_3_vals,best_3_indices = tf.nn.top_k(tf.math.negative(predicted_test_data ),3)
best_3_vals = tf.math.negative(best_3_vals)
best_3_vals,best_3_indices = map(lambda x: x.numpy(),[best_3_vals,best_3_indices])

In [ ]:
# plot results
visualizer.eval_soft_max_plot_test_whales(test_ds,best_3_indices,best_3_vals,columns=3,rows=40)
visualizer.eval_soft_max_plot_acc_by_class(test_df, best_3_indices, best_3_vals)